In [1]:
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [4]:
## Data Ingesion - from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [10]:
loader =WebBaseLoader("https://www.digitalocean.com/resources/articles/ai-customer-service")
loader

In [11]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.digitalocean.com/resources/articles/ai-customer-service', 'title': 'Implementing AI in Customer Service | DigitalOcean', 'description': 'Discover how AI customer service improves user experiences with faster, smarter, and more efficient support solutions.', 'language': 'en'}, page_content='Implementing AI in Customer Service | DigitalOceanBlogDocsGet SupportContact SalesDigitalOceanProductsFeatured ProductsDropletsScalable virtual machinesKubernetesScale more effectivelyAI / MLBuild and scale AI modelsCloudwaysManaged cloud hostingApp PlatformGet apps to market fasterComputeDropletsKubernetesCPU-Optimized DropletsFunctionsApp PlatformAI / MLGPU Droplets1-Click ModelsGenAI PlatformBare Metal GPUsBackups & SnapshotsBackupsSnapshotsSnapShooterNetworkingVirtual Private Cloud (VPC)Cloud FirewallsLoad BalancersDNSDDoS ProtectionManaged DatabasesMongoDBKafkaMySQLPostgreSQLCachingOpenSearchStorageSpaces Object StorageVolume Block StorageDeveloper Tool

In [12]:
# Load data -> docs -> Divide our text into chunks -> text -> vectors-> vector Embeddings -> Vector Store
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs = text_splitter.split_documents(docs)
docs


[Document(metadata={'source': 'https://www.digitalocean.com/resources/articles/ai-customer-service', 'title': 'Implementing AI in Customer Service | DigitalOcean', 'description': 'Discover how AI customer service improves user experiences with faster, smarter, and more efficient support solutions.', 'language': 'en'}, page_content='Implementing AI in Customer Service | DigitalOceanBlogDocsGet SupportContact SalesDigitalOceanProductsFeatured ProductsDropletsScalable virtual machinesKubernetesScale more effectivelyAI / MLBuild and scale AI modelsCloudwaysManaged cloud hostingApp PlatformGet apps to market fasterComputeDropletsKubernetesCPU-Optimized DropletsFunctionsApp PlatformAI / MLGPU Droplets1-Click ModelsGenAI PlatformBare Metal GPUsBackups & SnapshotsBackupsSnapshotsSnapShooterNetworkingVirtual Private Cloud (VPC)Cloud FirewallsLoad BalancersDNSDDoS ProtectionManaged DatabasesMongoDBKafkaMySQLPostgreSQLCachingOpenSearchStorageSpaces Object StorageVolume Block StorageDeveloper Tool

In [16]:
print(len(docs))

31


In [17]:
from langchain_openai import OpenAIEmbeddings

In [18]:
embeddings = OpenAIEmbeddings()

In [19]:
from langchain_community.vectorstores import FAISS

In [20]:
vectorstore_db = FAISS.from_documents(docs,embeddings)

In [21]:
vectorstore_db

In [22]:
query = "What is chatbots?"

In [24]:
result = vectorstore_db.similarity_search(query)
print(result[0].page_content)

Chatbots
Chatbots provide instant, round-the-clock responses to customer questions, reducing wait times and improving customer satisfaction when it comes to common queries. For businesses, they offer cost-effective support, enabling scalability by handling large volumes of inquiries, without hiring additional customer support agents. This improves operational efficiency and ensures a consistent level of service for customers, fostering loyalty and trust.
Sentiment analysis
Sentiment analysis helps businesses understand customer emotions and perceptions through their interactions. This AI technology scans customer feedback, detecting positive, negative, or neutral sentiments. For businesses, this offers valuable insights into customer satisfaction and areas for improvement, leading to informed decision-making. Customers benefit from a more empathetic, personalized service experience, as businesses can tailor their responses and services to better meet customer needs and expectations.


In [29]:
llm = ChatOpenAI(model="gpt-4o")

In [26]:
## Retrieval Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [32]:
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based on the provided context:
<context>
{context}
</context>
    """
)

In [33]:
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based on the provided context:\n<context>\n{context}\n</context>\n    '))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fc111078940>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc11107a260>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [35]:
from langchain_core.documents import Document
document_chain.invoke(
   {"input":"What is chatbots?",
    "context":[Document(page_content="What is chatbots and benefits of ai customer service?")],
    } 
)

'Chatbots are software applications designed to simulate human conversation, often through text or voice interactions. They use artificial intelligence (AI) to understand and respond to user queries, providing automated customer service and support. Chatbots can be integrated into websites, messaging apps, and customer service platforms to assist users with inquiries, complete transactions, or provide information.\n\nThe benefits of AI customer service, including chatbots, are numerous:\n\n1. **24/7 Availability**: Chatbots can operate around the clock, offering support to customers at any time, which enhances customer satisfaction and convenience.\n\n2. **Cost Efficiency**: Automating customer interactions reduces the need for a large customer service team, leading to cost savings for businesses.\n\n3. **Scalability**: Chatbots can handle multiple inquiries simultaneously, making it easier to manage high volumes of customer interactions without compromising service quality.\n\n4. **Co

#### However,we want the documents to first come from the retriever we just setup.That way,we can use the retriever to dynamically select the most relevant documnets and pass those in for a given question.

In [36]:
### Retriever
# Input -> Retriever -> vectorstoredb
vectorstore_db

In [37]:
retriever = vectorstore_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fc10c6c4d60>)

In [38]:
from langchain.chains import create_retrieval_chain

In [39]:
retriever_chain = create_retrieval_chain(retriever,document_chain)
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fc10c6c4d60>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based on the provided context:\n<context>\n{context}\n</context>\n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fc111078940>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc11107a260>, model

In [40]:
## Get the response from the LLM
response = retriever_chain.invoke({"input":"What is chatbots?"})
response['answer']

'AI customer service is a technology-driven approach that leverages artificial intelligence to manage and optimize customer interactions. It involves using machine learning, natural language processing, and other AI technologies to automate responses to customer inquiries, especially routine questions. This approach allows human agents to focus on more complex issues, enhancing the overall customer experience. Some key benefits of AI customer service include:\n\n1. **Instant and Round-the-Clock Responses**: AI can provide immediate responses to customer inquiries at any time, reducing wait times and improving customer satisfaction.\n\n2. **Cost-Effectiveness and Scalability**: AI can handle large volumes of inquiries without the need for additional hiring, making it a cost-effective solution for businesses looking to scale their customer service operations.\n\n3. **Consistent Service**: AI ensures a consistent level of service, which helps in building customer trust and loyalty.\n\n4. 

In [41]:
response

{'input': 'What is chatbots?',
 'context': [Document(metadata={'source': 'https://www.digitalocean.com/resources/articles/ai-customer-service', 'title': 'Implementing AI in Customer Service | DigitalOcean', 'description': 'Discover how AI customer service improves user experiences with faster, smarter, and more efficient support solutions.', 'language': 'en'}, page_content='Chatbots\nChatbots provide instant, round-the-clock responses to customer questions, reducing wait times and improving customer satisfaction when it comes to common queries. For businesses, they offer cost-effective support, enabling scalability by handling large volumes of inquiries, without hiring additional customer support agents. This improves operational efficiency and ensures a consistent level of service for customers, fostering loyalty and trust.\nSentiment analysis\nSentiment analysis helps businesses understand customer emotions and perceptions through their interactions. This AI technology scans customer

In [42]:
response['context']

[Document(metadata={'source': 'https://www.digitalocean.com/resources/articles/ai-customer-service', 'title': 'Implementing AI in Customer Service | DigitalOcean', 'description': 'Discover how AI customer service improves user experiences with faster, smarter, and more efficient support solutions.', 'language': 'en'}, page_content='Chatbots\nChatbots provide instant, round-the-clock responses to customer questions, reducing wait times and improving customer satisfaction when it comes to common queries. For businesses, they offer cost-effective support, enabling scalability by handling large volumes of inquiries, without hiring additional customer support agents. This improves operational efficiency and ensures a consistent level of service for customers, fostering loyalty and trust.\nSentiment analysis\nSentiment analysis helps businesses understand customer emotions and perceptions through their interactions. This AI technology scans customer feedback, detecting positive, negative, or